In [12]:
import requests, json
import pandas as pd
from bs4 import BeautifulSoup
import re
from difflib import SequenceMatcher
import urllib3

In [13]:
# Disable InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [14]:
# Define the brands you're interested in
brands = ['loreal', 'maybelline', 'lakme', 'mamaearth', 'pilgrim', 'mcaffeine', 'plum', 'clinique', 'swiss-beauty', 'blue-heaven', 'mars']
brands_complete = ['L\'Oreal Paris', 'Maybelline', 'Lakme', 'Mamaearth', 'Pilgrim', 'mCaffeine', 'Plum', 'Clinique', 'Swiss Beauty', 'Blue Heaven', 'MARS']

purplle_brands = {brands[i]: brands_complete[i] for i in range(len(brands))}
purplle_brands


{'loreal': "L'Oreal Paris",
 'maybelline': 'Maybelline',
 'lakme': 'Lakme',
 'mamaearth': 'Mamaearth',
 'pilgrim': 'Pilgrim',
 'mcaffeine': 'mCaffeine',
 'plum': 'Plum',
 'clinique': 'Clinique',
 'swiss-beauty': 'Swiss Beauty',
 'blue-heaven': 'Blue Heaven',
 'mars': 'MARS'}

In [15]:
# Define headers for requests
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

Scrape Purplle

In [16]:
# Extracted using big query from products_master table https://console.cloud.google.com/bigquery?sq=462171070671:1c1407b1a96b4f53ab1b47dbae178921
df = pd.read_csv('../../Data/products_data.csv')
df

,brand_name,name,our_price
0,Mamaearth,Mamaearth Natural Mosquito Repellent For Babie...,269.0
1,Mamaearth,Mamaearth After Bite Roll On For Rashes And Mo...,214.0
2,Mamaearth,Mamaearth Natural Repellent Mosquito Patches F...,171.0
3,Mamaearth,Mamaearth Natural Mosquito Repellent Gel (50 m...,85.0
4,Mamaearth,Mamaearth Anti Mosquito Fabric Roll On (8 ml),85.0
...,...,...,...
4660,Swiss Beauty,Swiss Beauty 3D Eyelashes Dream Lashes 03,244.0
4661,Swiss Beauty,Swiss Beauty Cover & Blend Stick Foundation 03...,314.0
4662,Swiss Beauty,Swiss Beauty Cover & Blend Stick Foundation 04...,314.0
4663,Swiss Beauty,Swiss Beauty Cover & Blend Stick Foundation 06...,314.0


In [17]:
def clean(string):
    # Remove special characters and spaces
    cleaned_string = re.sub(r'[^A-Za-z0-9]', '', string)
    cleaned_string = cleaned_string.lower()
    return cleaned_string

In [18]:
def scrape_purplle(brand):
    product_list = []

    df_brand = df[df['brand_name'] == purplle_brands[brand]]
    df_brand = df_brand.reset_index(drop=True)

    for index, row in df_brand.iterrows():
        product_list.append({'name': row['name'], 'price': row['our_price']})


    # url = f'https://www.purplle.com/brand/{brand}'
    # response = requests.get(url)
    # soup = BeautifulSoup(response.content, 'html.parser')

    # # Find all product elements based on the provided HTML structure
    # product_elements = soup.find_all('div', class_='pdt2')
    
    # for product_element in product_elements:
    #     product_name_element = product_element.find('div', class_='pro-name')
    #     product_price_element = product_element.find('span', class_='tx-0')

    #     if product_name_element and product_price_element:
    #         product_name = product_name_element.text.strip()
    #         product_price = product_price_element.text.strip()

    #         product_list.append({'name': product_name, 'price': product_price})

    return product_list

Scrape Myntra

In [19]:
def scrape_myntra_helper(brand, pageno):
    session = requests.Session()

    url = f'https://www.myntra.com/{brand}?p={pageno}'
    response = session.get(url, headers=headers, allow_redirects=False) 

    soup = BeautifulSoup(response.text, 'html.parser')

    script_element = soup.find_all('script', type='application/ld+json')
    required_data = None

    for script in script_element:
        json_data = json.loads(script.string)
        if json_data['@type'] == 'ItemList':
            required_data = json_data
            break

    product_list = []

    try:
        for item in required_data['itemListElement']: # none type error if the page doesn't have products
            s = requests.Session()
            res = s.get(item['url'], headers=headers, verify=False)

            soup = BeautifulSoup(res.text,"html.parser")

            script = None
            for s in soup.find_all("script"):
                if 'pdpData' in s.text:
                    script = s.get_text(strip=True)
                    break

            data = json.loads(script[script.index('{'):])
            name = data['pdpData']['name']
            price = discounted_price = data["pdpData"]["sizes"][0]["sizeSellerData"][0]["discountedPrice"]
            product_list.append({'name': name, 'price': price})
    except:
        pass
        
    return product_list    

In [20]:
def scrape_myntra(brand):
    product_list = []

    pgno = 1
    while True:
        prods = scrape_myntra_helper(brand, pgno)
        if prods == []:
            break
    
        product_list.extend(prods)
        pgno += 1

    return product_list

Compare Products and their Prices

In [41]:
# Function to clean and preprocess product names
def clean_product_name(name):
    # Remove formatting and symbols
    name = re.sub(r'[^a-zA-Z0-9]', '', name)
    # Convert to lowercase and remove extra spaces
    name = name.lower()
    return name

# Function to calculate the similarity between two product names
def get_name_similarity(name1, name2):
    return SequenceMatcher(None, name1, name2).ratio()

# Function to compare common products and their prices
def compare_products(brand, req_similarity_score):
    myntra_products = scrape_myntra(brand)
    purplle_products = scrape_purplle(brand)

    # print("Products found on myntra ", len(myntra_products))
    # print("Products found on purplle ", len(purplle_products))

    better_products = []
    products_done = {}

    for myntra_product in myntra_products:
        for purplle_product in purplle_products:
            myntra_name = clean_product_name(myntra_product['name'])
            purplle_name = clean_product_name(purplle_product['name'])

            myntra_price = int(myntra_product['price'])
            purplle_price = int(purplle_product['price'])

            similarity_score = get_name_similarity(myntra_name, purplle_name)

            if similarity_score > req_similarity_score and myntra_price < purplle_price and purplle_product['name'] not in products_done:  # You can adjust the threshold as needed
                better_products.append({
                    'myntra_name': myntra_product['name'],
                    'purplle_name': purplle_product['name'],
                    'myntra_price': myntra_product['price'],
                    'purplle_price': purplle_product['price'],
                    'similarity_score': similarity_score
                })
                products_done[purplle_product['name']] = True

    return better_products


In [42]:
import csv

# set as 0.965 for very very accurate results
similarity_score = 0.92
file_name = 'PriceScapeResults' if similarity_score > 0.95 else 'PriceScapeResults_LesserAccuracy'

with open(f'{file_name}.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['brand', 'myntra_name', 'purplle_name', 'myntra_price', 'purplle_price'])
    for brand in brands:
        common_products = compare_products(brand, similarity_score)
        for product in common_products:
            writer.writerow([brand, product['myntra_name'], product['purplle_name'], product['myntra_price'], product['purplle_price']])